In [15]:
import json

with open("whisper-v3-aligned-to-sw_all.json") as inf:
    data = json.load(inf)

In [14]:
def clean_text(text):
    if text.endswith("."):
        text = text[:-1]
    if text.endswith("!"):
        text = text[:-1]
    if text.endswith("?"):
        text = text[:-1]
    text = text.replace(",", "")
    return text.lower().strip()

In [22]:
def ucfirst(text):
    if len(text) == 0:
        return text
    return text[0].upper() + text[1:]

In [16]:
for item in data:
    data[item]["orig_clean"] = data[item]["orig"].replace("_", " ")
    if not "res" in data[item]:
        continue
    data[item]["res_clean"] = clean_text(data[item]["res"])
    if data[item]["orig_clean"] == data[item]["res_clean"]:
        data[item]["match"] = "simple"

In [24]:
for manual in ["1569"]:
    data[manual]["match"] = "norm"

for manual in ["0165"]:
    data[manual]["match"] = "spelling"
    data[manual]["corrected"] = data[manual]["orig"] + "."

for manual in ["2195", "2074"]:
    data[manual]["match"] = "spelling"
    data[manual]["corrected"] = ucfirst(data[manual]["orig_clean"])

In [19]:
pruned = {x: y for (x, y) in data.items() if not "match" in y}

In [20]:
pruned

{'2298': {'res': 'Arbetet innebär att ta emot mycket smärta och vanmakt.',
  'orig': 'arbetet innebär att ta emot mycken smärta och vanmakt',
  'orig_clean': 'arbetet innebär att ta emot mycken smärta och vanmakt',
  'res_clean': 'arbetet innebär att ta emot mycket smärta och vanmakt'},
 '4334': {'res': 'Hur kan vi vara så säkra på att ingen man ingår i besättningen, åtminstone som frekvent gäst?',
  'orig': 'kanske skyndar vi för fort ändå',
  'orig_clean': 'kanske skyndar vi för fort ändå',
  'res_clean': 'hur kan vi vara så säkra på att ingen man ingår i besättningen åtminstone som frekvent gäst'},
 '0165': {'res': 'ofta i form av uppläggningsavgift och avi-avgift.',
  'orig': 'ofta i form av uppläggningsavgift och aviavgift',
  'orig_clean': 'ofta i form av uppläggningsavgift och aviavgift',
  'res_clean': 'ofta i form av uppläggningsavgift och avi-avgift'},
 '0237': {'res': 'Då forcerar de dörren till bussen.',
  'orig': 'då forcerade de dörren till bussen',
  'orig_clean': 'då fo